In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-distractor-model")
tokenizer = T5Tokenizer.from_pretrained("t5-distractor-model")

/Users/mac/Projects/fine-tuning/T5-distractor-generation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def preprocess(example):
    correct_idx = ord(example["answer"]) - ord("A")
    correct = example["options"][correct_idx]
    distractors = [opt for i, opt in enumerate(
        example["options"]) if i != correct_idx]

    return {
        "input_text": f"generate distractors: question: {example['question']} context: {example['article']} answer: {correct}",
        "target_text": " | ".join(distractors)
    }

In [14]:
def generate_distractors(example, max_length=100000):
    # Use the same preprocessing logic as during training
    data = preprocess(example)
    inputs = tokenizer(data["input_text"],
                       return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split(" | ")  # Returns list of distractors

In [15]:
example = {
    "question": "In which country is the prom called a 'formal'?",
    "article": "It is well-known that the 'prom', a formal dance held at the end of high school or college, is an important date in every student's life. What is less well-known, however, is that in some countries, it goes by different names. For example, in Australia, it is called a 'formal'.",
    "options": ["USA", "UK", "Australia", "Canada"],
    "answer": "C"
}

distractors = generate_distractors(example)
print("Generated Distractors:", distractors)

Generated Distractors: ["generate distractors: question: In which country is the prom called a 'formal'? context: It is well-known that the 'prom', a formal dance held at the end of high school or college, is an important date in every student's life. What is less well-known, however, is that in some countries, it goes by different names. For example, in Australia, it is called a 'formal'. answer: Australia"]


In [16]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer


def preprocess(example):
    correct_idx = ord(example["answer"]) - ord("A")
    correct = example["options"][correct_idx]
    distractors = [opt for i, opt in enumerate(
        example["options"]) if i != correct_idx]

    return {
        "input_text": f"generate distractors: question: {example['question']} context: {example['article']} answer: {correct}",
        "target_text": " | ".join(distractors)
    }


def generate_distractors(example, model, tokenizer, max_length=64):
    model.eval()

    data = preprocess(example)
    input_text = data["input_text"]

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split(" | ")

In [18]:
example = {
    "question": "In which country is the prom called a 'formal'?",
    "article": "It is well-known that the 'prom', a formal dance held at the end of high school or college, is an important date in every student's life. What is less well-known, however, is that in some countries, it goes by different names. For example, in Australia, it is called a 'formal'.",
    "options": ["USA", "UK", "Australia", "Canada"],
    "answer": "C"
}

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-distractor-model")
tokenizer = T5Tokenizer.from_pretrained("t5-distractor-model")

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Generate distractors
distractors = generate_distractors(example, model, tokenizer)
print("Generated Distractors:", distractors)

Generated Distractors: ["generate distractors: question: In which country is the prom called a 'formal'? context: It is well-known that the 'prom', a formal dance held at the end of high school or college, is an important date in every student's life. What is"]


In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the fine-tuned model and tokenizer
model_path = "t5-distractor-model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Example input
question = "What is business analytics primarily used for?"
context = "Business analytics is the use of data, IT, statistics, and models to inform business decisions."
answer = "To make better decisions"

# Format the input as during training
input_text = f"generate distractors: question: {question} context: {context} answer: {answer}"

# Tokenize and prepare the input
input_ids = tokenizer.encode(
    input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

# Generate output
outputs = model.generate(input_ids=input_ids, max_length=64,
                         num_beams=5, early_stopping=True)

# Decode and print
distractors = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated distractors:", distractors)

Generated distractors: generate distractors: question: What is business analytics primarily used for? context: Business analytics is the use of data, IT, statistics, and models to inform business decisions. answer: To make better decisions.


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load fine-tuned model and tokenizer
model_path = "t5-distractor-model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Create an example input
question = "What is business analytics primarily used for?"
context = "Business analytics is the use of data, IT, statistics, and models to inform business decisions."
answer = "To make better decisions"

# Format input as used during training
input_text = f"generate distractors: question: {question} context: {context} answer: {answer}"

# Tokenize
inputs = tokenizer(input_text, return_tensors="pt",
                   truncation=True, max_length=512).to(device)

# Generate prediction
with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=64,
        num_beams=5,
        early_stopping=True
    )

# Decode and print
distractors = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated distractors:", distractors)

Generated distractors: To make better decisions


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-base"
checkpoint_path = "model_weights/t5-base-Race-Distractor-Generation-version0-step0.pt"
max_length = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(
    model_name, model_max_length=max_length)
tokenizer.add_special_tokens({"sep_token": "<sep>"})
model.resize_token_embeddings(32128)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(32128)

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model'])  # ⬅️ THIS is the fix
model.to(device)
model.eval()

question = "What is the main function of mitochondria in a cell?"
context = "Mitochondria are known as the powerhouses of the cell because they produce energy through cellular respiration. They convert glucose and oxygen into ATP, the energy currency of the cell."
answer = "To produce energy"
test_input = f"generate distractors: {question} context: {context} answer: {answer}"

inputs = tokenizer(test_input, return_tensors="pt",
                   truncation=True, padding=True).to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=64,
        num_beams=5,
        num_return_sequences=3,
        early_stopping=True
    )

# Decode and print
decoded = [tokenizer.decode(output, skip_special_tokens=True)
           for output in outputs]
print("Generated distractors:")
for d in decoded:
    print("-", d)

Generated distractors:
- False
- True
- Not_entailment


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(
    "potsawee/t5-large-generation-race-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "potsawee/t5-large-generation-race-QuestionAnswer")

context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him 
to play at Indian Wells and the Miami Open next month. The United States has extended 
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination 
will be required to enter the country until at least 10 April, but the Serbian has previously 
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country. 
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar 
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to 
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title 
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

inputs = tokenizer(context, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
question_answer = question_answer.replace(
    tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
question, answer = question_answer.split(tokenizer.sep_token)

print("question:", question)
print("answer:", answer)

question:  What is the best title for the passage?
answer:  Djokovic's application for special permission to enter the United States
